In [1]:
import WL as wl
import igraph as ig
import make_graph
import binvox_rw
import numpy as np
import torch
import networkx as nx
import grakel
import matplotlib.pyplot as plt
import make_graph_nx
import math
import random
import csv
import torch.nn as nn
import config as cfg
from evolution import WL_evolution,simple_evolution
from IPython.display import SVG, display
from mpl_toolkits.mplot3d import axes3d, Axes3D
from generate import SuperGenerator
from evolution import mutate
from utils import generate_z,slide,create_coords_from_voxels
from make_graph_nx import return_label
from kmeans_pytorch import kmeans
from make_cluster_graph_nx import convert_sparse_to_graph
from sklearn.cluster import KMeans

def load_file_data(container):
    for i in range(cfg.load_start,cfg.load_end+1,1):
        with open ('../DeepIE3D_Training/data/chair64_512/chair'+str(i)+'.binvox','rb') as f:
        #with open ('../DeepIE3D_Training/data/ex/sphere4.binvox','rb') as f:
            model=binvox_rw.read_as_3d_array(f)   #type..numpy array
            container.append(model.data)
    '''
    for i in range(cfg.load_start,cfg.load_end+1,1):
        with open ('../DeepIE3D_Training/data/plane64_512/plane'+str(i)+'.binvox','rb') as f:
        #with open ('../DeepIE3D_Training/data/ex/sphere4.binvox','rb') as f:
            model=binvox_rw.read_as_3d_array(f)   #type..numpy array
            container.append(model.data)
    '''
    return container

def make_binvox(data,filename):
    dense_data=binvox_rw.sparse_to_dense(data,[64,64,64])
    voxels=binvox_rw.Voxels(dense_data,[64,64,64], [0,0,0], 1, 'xyz')
    voxels.write('../DeepIE3D_Training/data/ex/'+filename+'.binvox')

def cluster_graph_sk(voxels):
    graphs=[]
    for i in range(len(voxels)):
        nonzero=torch.nonzero(torch.Tensor(voxels[i])>=0.5)
        count=torch.count_nonzero(nonzero)
        index=nonzero.to('cpu').detach().numpy()
        num_clusters=int(cfg.num_clusters*math.sqrt(count/cfg.std_num))
        km = KMeans(n_clusters=num_clusters,
            init='random',
            n_init=2,
            max_iter=10,
            tol=1e-04,
            random_state=0
            )
        km.fit_predict(index)
        cluster_centers=km.cluster_centers_
        graph=convert_sparse_to_graph(cluster_centers,np.array(cfg.center))
        graphs.append(graph)
    return graphs

def cluster_graph(voxels,counts):
    G=[]
    for i in range(len(voxels)):
        voxel_np=torch.from_numpy(voxels[i])
        nonzero=torch.nonzero(voxel_np==True)
        num_clusters=int(cfg.num_clusters*math.sqrt(counts[i]/cfg.std_num))
        cluster_ids_x, cluster_centers = kmeans(
            X=nonzero, num_clusters=num_clusters, distance='euclidean', device=torch.device('cpu')
        )
        sparse_np=cluster_centers.detach().numpy()
        make_binvox(
            torch.transpose(cluster_centers,0,1).detach().numpy(),'cluster='+str(num_clusters)+'index='+str(i+1))
        graph=convert_sparse_to_graph(sparse_np,np.array(cfg.center))
        G.append(graph)
    return G

def visgraph(G):
    for i in range(len(G)):
        svg = SVG(nx.nx_agraph.to_agraph(G[i]).draw(prog='fdp', format='svg'))
        display(svg)
    
def process_base_kernel(base_kernel):
    for i in range(len(base_kernel)):
        print('shape',base_kernel[i].X.shape)
        print(np.dot(base_kernel[i].X,base_kernel[i].X.T))
        
def count_voxel(voxels):
    counts=[]
    for i in range(len(voxels)):
        counts.append(np.count_nonzero(voxels[i]))
    return counts
'''
zs=[]
for i in range(9):
    zs.append(generate_z().to('cpu').detach().numpy().tolist())
print(WL_evolution([4],zs,SuperGenerator()))
'''
container=[] #True False
container2=[]
load_file_data(container)
load_file_data(container2)

G=cluster_graph_sk(container)
G2=cluster_graph_sk(container2)

G_grakel=grakel.graph_from_networkx(G, node_labels_tag='label')
G2_grakel=grakel.graph_from_networkx(G2,node_labels_tag='label')
gk=grakel.WeisfeilerLehman(n_iter=cfg.n_iter,normalize=True)
self=gk.fit(G_grakel)
print(self.X)
self=self.fit(G2_grakel)
print('K2',self.X)
#process_base_kernel(base_kernel)
#visgraph(G)





{0: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False), 1: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False), 2: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False)}
K2 {0: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False), 1: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False), 2: VertexHistogram(n_jobs=None, normalize=False, sparse='auto', verbose=False)}
